In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/transformers/7b-it/2/model.safetensors.index.json
/kaggle/input/gemma/transformers/7b-it/2/model-00003-of-00004.safetensors
/kaggle/input/gemma/transformers/7b-it/2/config.json
/kaggle/input/gemma/transformers/7b-it/2/gemma-7b-it.gguf
/kaggle/input/gemma/transformers/7b-it/2/model-00001-of-00004.safetensors
/kaggle/input/gemma/transformers/7b-it/2/tokenizer.json
/kaggle/input/gemma/transformers/7b-it/2/tokenizer_config.json
/kaggle/input/gemma/transformers/7b-it/2/model-00004-of-00004.safetensors
/kaggle/input/gemma/transformers/7b-it/2/special_tokens_map.json
/kaggle/input/gemma/transformers/7b-it/2/.gitattributes
/kaggle/input/gemma/transformers/7b-it/2/model-00002-of-00004.safetensors
/kaggle/input/gemma/transformers/7b-it/2/tokenizer.model
/kaggle/input/gemma/transformers/7b-it/2/generation_config.json
/kaggle/input/gemma/transformers/2b-it/2/model.safetensors.index.json
/kaggle/input/gemma/transformers/2b-it/2/gemma-2b-it.gguf
/kaggle/input/gemma/transformers/2

In [2]:
!pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq

In [3]:
import torch
from transformers import (
    AutoModelForCausalLM,  # Importing AutoModelForCausalLM for the causal language modeling task
    AutoTokenizer,  # Importing AutoTokenizer for tokenization
    BitsAndBytesConfig,  # Importing BitsAndBytesConfig for quantization configuration
    AutoConfig,  # Importing AutoConfig for model configuration
    set_seed  # Importing set_seed for reproducibility
)

# Setting seed for reproducibility
set_seed(42)

# Define the path to the model directory
MODEL_PATH = "/kaggle/input/deepseek-math"

# Configuration for quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Loading weights in 4-bit format
    bnb_4bit_quant_type="nf4",  # Using non-linear quantization with 4 bits
    bnb_4bit_compute_dtype=torch.bfloat16,  # Using bfloat16 for computation
    bnb_4bit_use_double_quant=True  # Using double quantization
)

# Load the model configuration
config = AutoConfig.from_pretrained(MODEL_PATH)
config.gradient_checkpointing = True  # Enable gradient checkpointing for memory optimization

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Load the pre-trained model with specific configurations
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",  # Automatically choose device placement
    torch_dtype="auto",  # Automatically choose torch data type
    trust_remote_code=True,  # Trust remote code for loading model
    config=config  # Set the loaded configuration
    # quantization_config=quantization_config,  # Optionally, enable quantization with specified configuration
)

2024-04-16 13:05:12.533013: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 13:05:12.533143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 13:05:12.705641: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
model.dtype  # Check the data type of the model parameters
import pandas as pd
from tqdm import tqdm

# Flag to indicate whether the dataset is private or not
PRIVATE = True

# Read the test dataset
df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/test.csv')
df.head()

,id,problem
0,000aaa,What is $1-1$?
1,111bbb,What is $0\times10$?
2,222ccc,Solve $4+x=4$ for $x$.


In [5]:
# If the test dataset is empty, switch to the training dataset and set PRIVATE flag accordingly
if len(df) < 5:
    df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
    PRIVATE = False
df.head()

,id,problem,answer
0,229ee8,"Let $k, l > 0$ be parameters. The parabola $y ...",52
1,246d26,Each of the three-digits numbers $111$ to $999...,250
2,2fc4ad,Let the `sparkle' operation on positive intege...,702
3,430b63,What is the minimum value of $5x^2+5y^2-8xy$ w...,800
4,5277ed,There exists a unique increasing geometric seq...,211


In [6]:
# Perform garbage collection to free up memory
import gc

# Set the device for computation
device = 'cuda'  # Using CUDA for GPU acceleration

In [7]:
def naive_parse(answer):
    """
    Naive parsing function to extract numerical values from a string.
    
    Args:
    - answer (str): Input string containing numerical values.
    
    Returns:
    - str: Extracted numerical values as a string.
    """
    out = []  # Initialize an empty list to store extracted numerical values
    start = False  # Flag to indicate the start of a numerical value
    end = False  # Flag to indicate the end of a numerical value
    for l in reversed(list(answer)):  # Iterate over characters in reverse order
        if l in '0123456789' and not end:  # Check if the character is a digit and not already at the end
            start = True  # Set the start flag to True
            out.append(l)  # Append the digit to the output list
        else:
            if start:  # If the start flag is True
                end = True  # Set the end flag to True
        
    out = reversed(out)  # Reverse the output list to get the original numerical value
    return ''.join(out)  # Convert the list of digits to a string and return
import transformers

# Initialize a text generation pipeline using the provided model and tokenizer
pipeline = transformers.pipeline(
    "text-generation",  # Specify the task as text generation
    model=model,  # Provide the pre-trained model
    tokenizer=tokenizer,  # Provide the tokenizer for processing inputs
    torch_dtype='auto',  # Automatically choose torch data type
    device_map="auto",  # Automatically choose device placement
)

In [8]:
print(f"Transformers Version: {transformers.__version__}")
import torch

# Disable memory-efficient sparse tensors for CUDA operations
torch.backends.cuda.enable_mem_efficient_sdp(False)

Transformers Version: 4.39.3


In [9]:
import re
import sys
import subprocess

def process_output(output):
    """
    Process the output of a mathematical expression.
    
    Args:
    - output (str): Output containing mathematical expressions and possibly code blocks.
    
    Returns:
    - Tuple[int, int]: Processed outputs from mathematical expressions and code blocks.
    """
    result = output
    
    try:
        # Extract code from the output
        code = output.split('```')[1][7:]

        # Write the extracted code to a file
        with open('code.py', 'w') as fout:
            fout.write(code)

        # Execute the code and capture the output
        batcmd = 'timeout 7 ' + sys.executable + ' code.py'
        try:
            shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
            print(shell_output)
            code_output = round(float(eval(shell_output))) % 1000
        except:
            code_output = -1

        print('CODE RESULTS', code_output)
    
    except Exception as e:
        print(e)
        print('ERROR PARSING')
        code_output = -1
    
    try:
        # Find boxed expressions in the output
        result_output = re.findall(r'\\boxed\{(.*)\}', result)

        print('BOXED', result_output)
        # If no boxed expression is found, try naive parsing
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]

        print('BOXED', result_output)
        # If no result is obtained from parsing, set to -1
        if not len(result_output):
            result_output = -1
        
        else:
            # Evaluate the expression and take modulo 1000
            result_output = round(float(eval(result_output))) % 1000
    
    except Exception as e:
        print(e)
        print('ERROR PARSING')
        result_output = -1
    
    return result_output, code_output

In [10]:
import re
from collections import defaultdict

# Instruction for the tool user
tool_instruction = " The answer should be given as a non-negative modulo 1000."
tool_instruction += '\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}.'

# Determine the number of repetitions based on whether the dataset is private
n_repetitions = 8 if PRIVATE else 2

# Lists to store results and answers
total_results = []
total_answers = []

# Iterate over the dataset
for i in tqdm(range(len(df))):
    id_ = df['id'].loc[i]
    problem = df['problem'].loc[i]
    
    messages = [
        {
            "role": "user", 
            "content": problem + tool_instruction
        }
    ]
    
    query_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False
    )
    
    results = []
    answers = []
    
    # Perform multiple repetitions for robustness
    for _ in tqdm(range(n_repetitions)):
        try:
            # Generate output using the text generation pipeline
            raw_output = pipeline(
                query_prompt, 
                max_new_tokens=2048, 
                do_sample=True, 
                temperature=0.89645,
                return_full_text=False
            )
            raw_output = raw_output[0]['generated_text']

            # Process the output to extract results and code outputs
            result_output, code_output = process_output(raw_output)

            # Clear CUDA memory and perform garbage collection
            torch.cuda.empty_cache()
            gc.collect()

        except Exception as e:
            print(e)
            result_output, code_output = -1, -1
        
        results.append(result_output)
        answers.append(code_output)
    
    total_results.append(results)
    total_answers.append(answers)

  0%|          | 0/2 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


77/2

CODE RESULTS 38
BOXED ['988']
BOXED 988



 50%|█████     | 1/2 [00:36<00:36, 36.83s/it]Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


2*(Mod(16 + (k - sqrt(k*(k - l + 4)))**2/(2*k**2) + (k + sqrt(k*(k - l + 4)))**2/(2*k**2), 500))

CODE RESULTS -1
BOXED ['421']
BOXED 421



  0%|          | 0/2 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


...


  0%|          | 0/2 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Traceback (most recent call last):
  File "/kaggle/working/code.py", line 29, in <module>
    result = find_f_of_100()
  File "/kaggle/working/code.py", line 25, in find_f_of_100
    f_100 = solve(Eq(f_f_f_100, f_100), f_n)[0]
IndexError: list index out of range


CODE RESULTS -1
BOXED ['3', '3', '3']
BOXED 3



 50%|█████     | 1/2 [01:07<01:07, 67.02s/it]Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


347

CODE RESULTS 347
BOXED ['801']
BOXED 801



100%|██████████| 10/10 [11:35<00:00, 69.52s/it]


In [11]:
import numpy as np
from collections import Counter

# List to store final answers
final_answers = []

# Iterate over the pairs of total_answers and total_results
for a, b in zip(total_answers, total_results):
    # Convert lists to numpy arrays
    a = np.array(a)
    b = np.array(b)
    
    # Replace negative values in 'a' with corresponding values from 'b'
    a[a < 0] = b[a < 0]
    
    # Count occurrences of each answer and find the most common one
    pred = Counter(a.tolist()).most_common(2)

    # Select the most common answer, excluding negative values if possible
    ans = pred[0][0] if not pred[0][0] < 0 else pred[1][0]

    # Append the final answer to the list
    final_answers.append(ans)
    print(ans)

38
395
176
800
83
24
848
256
918
3


In [12]:
# Assign the final answers to the 'answer' column in the DataFrame
df['answer'] = final_answers
# Display the DataFrame with the updated 'answer' column
df

,id,problem,answer
0,229ee8,"Let $k, l > 0$ be parameters. The parabola $y ...",38
1,246d26,Each of the three-digits numbers $111$ to $999...,395
2,2fc4ad,Let the `sparkle' operation on positive intege...,176
3,430b63,What is the minimum value of $5x^2+5y^2-8xy$ w...,800
4,5277ed,There exists a unique increasing geometric seq...,83
5,739bc9,For how many positive integers $m$ does the eq...,24
6,82e2a0,Suppose that we roll four 6-sided fair dice wi...,848
7,8ee6f3,"The points $\left(x, y\right)$ satisfying $((\...",256
8,bedda4,Let $ABCD$ be a unit square. Let $P$ be the po...,918
9,d7e9c9,A function $f: \mathbb N \to \mathbb N$ satisf...,3


In [13]:
# Save the DataFrame with 'id' and 'answer' columns to a CSV file
df[['id', 'answer']].to_csv("submission.csv", header=True, index=False)
# Display the first few rows of 'id' and 'answer' columns
df[['id', 'answer']].head()

,id,answer
0,229ee8,38
1,246d26,395
2,2fc4ad,176
3,430b63,800
4,5277ed,83


In [14]:
# If the dataset is not private, evaluate model performance on the training data
if not PRIVATE:
    # Load the training dataset
    df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
    
    # Assign model predictions to the 'model_answer' column
    df['model_answer'] = final_answers
    
    # Check if the model predictions match the ground truth answers
    df['match'] = df.answer == df.model_answer
    
    # Calculate and print the number of matches
    print(f'{df.match.sum()} matches in {len(df)} examples')

1 matches in 10 examples


In [15]:
# Write a simple Python script to a file
with open('code.py', 'w') as fout:
    fout.write("print('done')")

# Execute the script with a timeout using subprocess
batcmd = 'timeout 7 ' + sys.executable + ' code.py'
try:
    # Capture and print the output of the script
    shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
    print(shell_output)
except:
    pass  # Ignore any errors during execution

done

